In [ ]:
pip install torch torchvision pillow

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import os
import numpy as np

In [3]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512 * 512),
            nn.Tanh()  # Output in range [-1, 1]
        )
        
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 512, 512)  # Reshape to image dimensions
        return img

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(512 * 512, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Flatten the image
        validity = self.model(img_flat)
        return validity

# Initialize Generator and Discriminator
latent_dim = 100
generator = Generator(latent_dim)
discriminator = Discriminator()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator.to(device)
discriminator.to(device)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=262144, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [5]:

def load_images(image_dir):
    # List all image files while ignoring non-image files like .DS_Store
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    # Open, resize, and convert images
    images = [Image.open(img).resize((512, 512)).convert('L') for img in image_paths]

    # Transform images to PyTorch tensors and normalize to [-1, 1]
    transform = transforms.Compose([
        transforms.Grayscale(),  # Ensure single channel
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
    ])
    
    # Apply transformations to all images
    images = [transform(img) for img in images]

    return images

# Load your images
image_dir = 'images/images'
train_images = load_images(image_dir)
train_loader = torch.utils.data.DataLoader(train_images, batch_size=64, shuffle=True)

In [8]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Binary Cross-Entropy Loss
class BCEWithLogitsLoss(nn.Module):
    def __init__(self):
        super(BCEWithLogitsLoss, self).__init__()
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, y_pred, y_true):
        return self.loss(y_pred, y_true)

# Initialize the loss function
criterion = BCEWithLogitsLoss()


# Define the optimizers
# Define the optimizers with new parameters
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [9]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for i, imgs in enumerate(train_loader):
        
        # Move images to device
        imgs = imgs.to(device)
        
        # Labels
        valid = torch.ones(imgs.size(0), 1).to(device)
        fake = torch.zeros(imgs.size(0), 1).to(device)
        
        # ---------------------
        #  Train Generator
        # ---------------------
        optimizer_G.zero_grad()
        
        # Sample random noise as generator input
        z = torch.randn(imgs.size(0), latent_dim).to(device)
        
        # Generate images
        gen_imgs = generator(z)
        
        # Loss for generator: aim to fool the discriminator
        g_loss = criterion(discriminator(gen_imgs), valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        
        # Loss for real images
        real_loss = criterion(discriminator(imgs), valid)
        
        # Loss for fake images
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()

    print(f"[Epoch {epoch+1}/{num_epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save generated images at checkpoints
    if (epoch+1) % 100 == 0:
        save_image(gen_imgs.data[:25], f"Dataset_copy/epoch_{epoch+1}.png", nrow=5, normalize=True)

[Epoch 1/50] [D loss: 0.6935135126113892] [G loss: 0.6924383044242859]
[Epoch 2/50] [D loss: 0.6939287185668945] [G loss: 0.6916404366493225]
[Epoch 3/50] [D loss: 0.6936917304992676] [G loss: 0.6921124458312988]
[Epoch 4/50] [D loss: 0.6931482553482056] [G loss: 0.6931451559066772]
[Epoch 5/50] [D loss: 0.6931817531585693] [G loss: 0.6930780410766602]
[Epoch 6/50] [D loss: 0.6931490898132324] [G loss: 0.6931434273719788]
[Epoch 7/50] [D loss: 0.6931750774383545] [G loss: 0.6930917501449585]
[Epoch 8/50] [D loss: 0.6931480169296265] [G loss: 0.6931455135345459]
[Epoch 9/50] [D loss: 0.6931474208831787] [G loss: 0.6931467056274414]
[Epoch 10/50] [D loss: 0.6931513547897339] [G loss: 0.693138837814331]
[Epoch 11/50] [D loss: 0.6931473612785339] [G loss: 0.693146824836731]
[Epoch 12/50] [D loss: 0.6931490898132324] [G loss: 0.6931432485580444]
[Epoch 13/50] [D loss: 0.6931475400924683] [G loss: 0.6931465268135071]
[Epoch 14/50] [D loss: 0.6931474208831787] [G loss: 0.6931467056274414]
[Ep

In [10]:
# Latent vectors for interpolation
latent_vector_1 = torch.randn(1, latent_dim).to(device)  # First latent vector
latent_vector_2 = torch.randn(1, latent_dim).to(device)  # Second latent vector

# Interpolation function
def interpolate_latents(latent_1, latent_2, alpha):
    return (1 - alpha) * latent_1 + alpha * latent_2

# Generate and save intermediate images
alpha_values = np.linspace(0, 1, 10)  # 10 interpolated images
for alpha in alpha_values:
    # Create a batch of 2 latent vectors by interpolating and concatenating
    latent_interpolated = torch.cat([
        interpolate_latents(latent_vector_1, latent_vector_2, alpha), 
        interpolate_latents(latent_vector_2, latent_vector_1, alpha)
    ], dim=0)
    
    # Pass the batch through the generator
    generated_images = generator(latent_interpolated)
    
    # Save both generated images
    save_image(generated_images.data, f"Dataset_copy/interpolated_images_alpha_{int(alpha * 100)}.png", nrow=2, normalize=True)


FileNotFoundError: [Errno 2] No such file or directory: 'Dataset_copy/interpolated_images_alpha_0.png'